In [33]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
from Base_BdTopo.Import_outils import *
from Base_BdTopo.Rond_points import *
from Base_BdTopo.Troncon_base import *
from Base_BdTopo.Troncon_elementaire import *
from Base_BdTopo.Gestion_2_chaussee import *
from Base_BdTopo.Regroupement_correspondance import *
from Import_trafics import Comptage
import Connexion_Transfert as ct
import os
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize, linemerge,unary_union
from shapely.geometry import Point,MultiLineString, LineString
from shapely import speedups
from collections import Counter
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import Table, Column, Integer, String, MetaData
from sqlalchemy.sql import select
from sklearn.cluster import DBSCAN
import Outils
import psycopg2
#pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 50)
#pd.set_option('display.max_colwidth', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [188]:
speedups.disable()

In [3]:
#param de sauvegarde temporaire : 
def save_fichier(gdf,nom_fichier):
    gdf.to_file(os.path.join(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\OTV\17',nom_fichier+'.shp'))

# A FAIRE : 
- "triangle" multiple : TRONROUT0000000032823937
- prise en compte des petits troncon qui coupe perpendiculairement les 2*2voies sans autres tronconcs dans la continuité TRONROUT0000000032846678
- mettre en forme les fonctions, regroupés dans des modules si besoin

# AVANCEMENT : 
au 26/09 : les différentes parties jusqu'au chapitre 4.4 sont opérationnelles, mais des erreurs persistent. **Cependant c'est suffisant pour localiser les tronçons elementaires et vérifier si plusieurs point de comptages sont localisés sur le même troncon elementaire.**
<br> Le chapitre 4.5 est un premier essai pour faire un graph à partir des tronc elementaires. C'est à vérifier / reprendre / améliorer.

# IMPORTER LES DONNEES 
> depuis la base de données locale, en ne prenant en compte que les lignes qui intersectent une zone carré au N-O

In [3]:
df=import_donnes_base('gti_otv_pg11','public', 'traf2019_bdt17_ed19_l','traf2019_bdt17_ed19_l_vertices_pgr')
df2_chaussees=df.loc[df.nature.isin(['Type autoroutier', 'Route à 2 chaussées'])]

c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\sqlalchemy\dialects\postgresql\base.py:3132: SAWarning: Did not recognize type 'xml' of column 'metadata'
  util.warn(
c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\sqlalchemy\dialects\postgresql\base.py:3132: SAWarning: Did not recognize type 'xml' of column 'fileid'
  util.warn(
c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\sqlalchemy\dialects\postgresql\base.py:3132: SAWarning: Did not recognize type 'xml' of column 'parentid'
  util.warn(


# 1 RONDS POINTS

In [4]:
df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=identifier_rd_pt(df)
df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index

# 2. DESCCRIPTION DETAILLEE DES TRAITEMENTS 
> **Pour effectuer le traitements réellement passer au chapitre 3** <br> La base c'est : 
- regrouper les lignes qui se suivent
- trouver les lignes d'arret
- trouver les lignes suivantes
- voir le cas des lignes suivantes : 
    - rd point
    - mm route qui se sépare
    - rien

><br> L'idée c'est ça  : on met la premiere ligne a tester dans une liste, on l'utilise et on stocke les lignes associées dans une autre liste. ensuite on check les lignes suivantes : si elles sont dans un cas où il faut continuer les traitements, on remplace la ligne de départ dans la liste de départ par ces nouvelles lignes, pour lesquelles on applique de nouveau la recherche des lignes qui se suivent. tant que la liste des lignes suivantes est pa vide on reste dans une boucle while

## 2.1 Troncons de base

In [75]:
# ligne a tester : TRONROUT0000000339411609 TRONROUT0000000032869130 TRONROUT0000000032859410
id_lignes='TRONROUT0000000032999215'
#trouver tous les troncons proches qui se suivent
#list_troncon=[id_ligne]+[a for a in at.liste_troncon_base(id_ligne,df_lignes)]
list_troncon=liste_complete_tronc_base(id_lignes,df_lignes,[])

## 2.2 lignes d'arret

In [76]:
#trouver les troncons de debut et de fin :ceux pour lesquels on a un nombre de ligne sur target source >2
dico_deb_fin=deb_fin_liste_tronc_base(df_lignes, list_troncon)

In [77]:
dico_deb_fin

{0: {'id': 'TRONROUT0000000353890775',
  'type': 'source',
  'num_node': 1294226,
  'nature': 'Route à 1 chaussée',
  'geom_node': 'POINT(407733.4 6506844.7)',
  'voie': 'D243',
  'codevoie': '173100050'},
 1: {'id': 'TRONROUT0000000032999200',
  'type': 'target',
  'num_node': 380168,
  'nature': 'Route à 1 chaussée',
  'geom_node': 'POINT(406903.3 6507385)',
  'voie': 'D243',
  'codevoie': '173100050'}}

In [78]:
ligne='TRONROUT0000000353890775'
for v in dico_deb_fin.values() : 
    if v['id']==ligne :
        voie=v['voie']
        noeud=v['num_node']
        geom_noeud=v['geom_node']
        type_noeud=v['type']
        codevoie=v['codevoie']
        nature=v['nature']
        break
    else : 
        continue

## 2.3 lignes suivantes

In [79]:
#vérifier si une ligne touche un rd point
# aprtir d'une ligne et d'un numero de noeud, on récupère les lignes qui touchent
lignes_adj=df_lignes.loc[((df_lignes['source']==noeud)|(df_lignes['target']==noeud))&(df_lignes.index!=ligne)]

In [80]:
lignes_adj

,id,nature,nom_1_g,nom_1_d,numero,importance,cl_admin,gestion,fictif,largeur,nb_voies,sens,etat,inseecom_g,inseecom_d,codevoie_g,codevoie_d,urbain,vit_moy_vl,restr_p,dept,dept_2019,dept_2018,dept_2017,dept_2016,...,vts_type_pl,src_vma,vma_vl,vma_pl,vma_type,codau_cont,tmja_cont,pc_pl_cont,source,target,recup,attr_modif,geom,ann_pc_pl,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt_entrant,codevoie_rdpt_entrant,nb_obj_sig_entrant,valeur_sens,id_ign_entrant
id_ign,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TRONROUT0000000353890776,1481737,Route à 1 chaussée,RTE DU CHAILLAUD,RTE DU CHAILLAUD,None,4,Autre,None,Non,3.0,1.0,Sens direct,En service,17310,17310,173100050,173100050,Oui,40.0,NaN,17,17,17,17,17,...,estimee,osm,50.0,50,verifiee,None,NaN,NaN,1294226,1259277,pk_ign,None,"MULTILINESTRING ((407733.400 6506844.700, 407733.300 6506839.600, 407733.400 6506830.700, 407733.000 6506824.700))",None,3,POINT(407733.4 6506844.7),3,POINT(407733 6506824.7),NaN,NaN,NaN,NaN,NaN,NaN,NaN
TRONROUT0000000333968675,1454788,Route à 1 chaussée,RTE DU CHAILLAUD,RTE DU CHAILLAUD,D243,4,Départementale,Charente-Maritime,Non,3.0,1.0,Sens direct,En service,17310,17310,173100050,173100050,Oui,40.0,NaN,17,17,17,17,17,...,estimee,osm,50.0,50,verifiee,None,NaN,NaN,380147,1294226,pk_ign,None,"MULTILINESTRING ((407748.100 6506827.600, 407740.300 6506838.700, 407733.400 6506844.700))",None,3,POINT(407748.1 6506827.6),3,POINT(407733.4 6506844.7),NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.4 cas particulier 

### 2.4.1 rd pt

In [81]:
#verfier si une ligne rouche un rd point et recup le numero si c'est le cas
check_rdpt, num_rdpt=verif_touche_rdpt(lignes_adj)
if check_rdpt : 
    lignes_rdpt, lignes_sortantes=recup_lignes_rdpt(carac_rd_pt,num_rdpt,list_troncon,voie,codevoie)

### 2.4.2 mm route qui se sépare

In [181]:
recup_route_split('TRONROUT0000000353890775',list_troncon,voie,codevoie, lignes_adj,noeud,geom_noeud, type_noeud, nature,df_lignes)

['TRONROUT0000000353890776', 'TRONROUT0000000333968675']

### 2.4.3 trouver les routes coinces entre 2 voies qui se sépare -<|
 ce sont des lignes qui  dont : 
- si une des lignes adjacente à le mm nom de voie
- on prend l'autre si elle a un sens direct ou inverse
- on recherche son source ou target different du num_node de dico_deb_fin
- on cherche les lignes qui touche ce noeud, qui ont le mme nom de voie et un sens direct ou inverse
- on recupere cette ligne, et le num de noeud differnts de celui de la 1ere ligne qui bifurque
- si ce num de noeud correspond à la ligne adj source ou target, alors cette ligne adj source ou target doit être ajoutée à la liste

In [49]:
recup_triangle(ligne,voie, codevoie,lignes_adj, noeud,geom_noeud, type_noeud, df_lignes)

[]

## 2.5 route à 2 chaussées
L'idée c'est si la ligne est une route à 2 chaussée : 
- on prend l'ensemble du troncon elementaire
- on agerge les lignes
- on recupere le centre
- on recupere la longueur
- on pourrait chercher à comparer les bouding box

In [53]:
#dans le cas où les longeuers ne coincident pas
#recuperer la ligne proche
ligne_proche, ligne_filtres, longueur_base=trouver_chaussees_separee(list_troncon, df_avec_rd_pt)

In [54]:
ligne_proche

'TRONROUT0000000032840410'

In [55]:
gestion_voie_2_chaussee(list_troncon, df_avec_rd_pt, 'TRONROUT0000000032840659',carac_rd_pt )[0]

['TRONROUT0000000032843157',
 'TRONROUT0000000121811702',
 'TRONROUT0000000032840410',
 'TRONROUT0000000342863179',
 'TRONROUT0000000032843162',
 'TRONROUT0000000032843170',
 'TRONROUT0000000032843156']

# Tests de deroulement global

In [183]:
lignes_troncon_elem(df_avec_rd_pt,carac_rd_pt,'TRONROUT0000000353890775',[])

['TRONROUT0000000032999202',
 'TRONROUT0000000032999200',
 'TRONROUT0000000032999198',
 'TRONROUT0000000273363048',
 'TRONROUT0000000032999215',
 'TRONROUT0000000353890775',
 'TRONROUT0000000032999158',
 'TRONROUT0000000353890776',
 'TRONROUT0000000032999196',
 'TRONROUT0000000333968675']

# 3. Boucler sur 'ensembl des lignes
On va faire tourner les traitements sur l'ensemble des lignes, en partant de la liste des id_ign, pour chaque ligne : 
- si la ligne ne remplit pas les critères d'exclusion on continu, sinon on passe à la suivante
- fonction de recherche des lignes associées
- df des association
- filtre de la liste des id_ign de base

In [184]:
#traiter les bretelles en 1er, puis traiter par longueur decroissante
bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']=='Bretelle'].id_ign.tolist()
sans_bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']!='Bretelle'].copy()
list_sans_bretelle=sans_bretelle.id_ign.tolist()
list_tri_longueur=sans_bretelle.loc[sans_bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
list_id_ign=bretelle+list_tri_longueur

In [ ]:
df_affectation, dico_erreur, lignes_traitees, lignes_non_traitees=regrouper_troncon(
    list_id_ign, df_avec_rd_pt, carac_rd_pt,df2_chaussees,[])

In [205]:
export=df_avec_rd_pt.merge(df_affectation, left_on='id_ign',right_on='id')[[
    'id_ign','numero','codevoie_d','geom','idtronc']].to_file(r'D:\Boulot\AffairesEnCours\OTV\17\temp_linearisation\troncelem2019_Bdt17_ed19_l.shp')

In [5]:
#df_affectation=gp.read_file(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\OTV\17\temp_linearisation\troncelem2019_Bdt17_ed19_l.shp')

In [12]:
df_affectation#.rename(columns={id:'id'}, inplace=True)

,id,numero,codevoie_d,idtronc,geometry
0,TRONROUT0000000032889317,D129,None,13634,"LINESTRING (438601.800 6549651.500, 438647.200..."
1,TRONROUT0000000032912378,D114,None,11125,"LINESTRING (412903.300 6542032.700, 412899.200..."
2,TRONROUT0000000032916358,D18,None,11213,"LINESTRING (420030.400 6540991.200, 420028.100..."
3,TRONROUT0000000032918012,D224,None,5738,"LINESTRING (450318.700 6539202.400, 450315.100..."
4,TRONROUT0000000032918239,D229E2,170710053,1836,"LINESTRING (440387.800 6539813.200, 440384.300..."
...,...,...,...,...,...
112049,TRONROUT0000000033016335,D251E5,173260210,23599,"LINESTRING (443809.900 6493763.300, 443726.200..."
112050,TRONROUT0000000033019930,D249,172950084,72219,"LINESTRING (436368.500 6490858.500, 436391.700..."
112051,TRONROUT0000000033035833,None,None,28355,"LINESTRING (436202.700 6474152.100, 436178.900..."
112052,TRONROUT0000002002803401,D238,170360592,34208,"LINESTRING (389754.600 6537930.100, 389736.300..."


## 3.1 EXEMPLE SUR UNE LIGNE

In [191]:
df_affectation2, dico_erreur2, lignes_traitees2, lignes_non_traitees2=regrouper_troncon(
    ['TRONROUT0000000353890775'], df_avec_rd_pt, carac_rd_pt,df2_chaussees,[])
df_affectation2

0 2020-12-08 19:10:20.216625 nb lignes traitees : 0


,id,idtronc
0,TRONROUT0000000032999202,0
1,TRONROUT0000000032999200,0
2,TRONROUT0000000032999198,0
3,TRONROUT0000000273363048,0
4,TRONROUT0000000032999215,0
5,TRONROUT0000000353890775,0
6,TRONROUT0000000032999158,0
7,TRONROUT0000000353890776,0
8,TRONROUT0000000032999196,0
9,TRONROUT0000000333968675,0


## 3.2 CORRECTIONS DES CAS PARTICULIERS

In [112]:
#creer la synth des trcon_elem
tronc_elem_synth=carac_te(df_avec_rd_pt, df_affectation)
#creer la coquille qui va etre mise à jour
df_affectation_upd=creer_MaJ_te(df_affectation)

### 3.2.1 Bretelles

In [113]:
# table de corespondance
corresp_bret=corresp_te_bretelle('gti_otv_pg11',df_lignes, tronc_elem_synth, 75)
#mise à jour
MaJ_te(df_affectation_upd, corresp_bret, 'Bretelle')

c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


### 3.2.2 arrivée de rd point isolée
exemple : TRONROUT0000000112535635

In [114]:
corresp_entree_rdpt=corresp_te_arrive_rdpt(carac_rd_pt,df_avec_rd_pt,tronc_elem_synth)
#mise à jourMfectation_upd, corresp_entree_rdpt, 'entree_rd_pt')

### 3.2.3 petit troncon
- la df de correspondance est faite, il faut maintenant fair les corrections et pouvoir les tracer

In [ ]:
corresp_petit_tronc=corresp_petit_tronc(df_lignes,df_affectation,tronc_elem_synth)

In [121]:
#mise à jour
MaJ_te(df_affectation_upd, corresp_petit_tronc, 'petit_troncon')

## 3.3 4.4 Maj tronc_elem finale
A VERIFIER

In [122]:
df_lignes_fin=df_lignes.reset_index().copy()
if not 'id_tronc_elem' in df_lignes_fin.columns : 
    df_lignes_fin=df_lignes_fin.merge(df_affectation_upd[['id','idtronc_y']], left_on='id_ign', right_on='id').rename(columns={'idtronc_y':'id_tronc_elem'})
else : 
    df_lignes_fin.update(df_affectation_upd.reset_index().set_index('id').rename(columns={'idtronc_y':'id_tronc_elem'}))

In [124]:
#export
df_lignes_fin[['geom', 'id_ign',  'id_tronc_elem','id_rdpt',
       'nb_rte_rdpt', 'nb_obj_sig_entrant']].reset_index().to_file(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\OTV\17\temp_linearisation\troncelem2019_Bdt17_ed169_l_final.shp')

# 4. Creer graph (EN CHANTIER)
- en premier on traite les cas particuliers  : beretelles, petits troncons, arrivé de rd point isolée
- on cerche a creer une df avec tronc_elem|tronc_ign|src_elem|tgt_elem
- il faudrait un filtre sur les petits troncon entre 2 grands et les associer aux grands
- il faudrait regrouper les bretelles aussi

## 4.5 sources et target regroupes

In [28]:
def creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation ):
    """
    associer des tronc elem a des noeud représentatif pour créer un graph
    en entree : 
        df_lignes df des lignes bdtopo avec rd point et index=id_ign
        ligne_base : df des lignes constituant le tronc elem. issus de df_lignes
        df_affectation : df de correspondance id_ign idtronc_elem, issu de regrouper_troncon
    en sortie : 
        noeuds_fin : df avec index = noeud_unique (int) et idtronc)=tuple des idtroncelem qui touchent ce noeud
        coresp_te_noeud : toute les correspondances noeud-lignes
        noeud_uniq : list edes noeud de fin de tronc elem
    """
    noeud_uniq=[k for k,v in Counter(ligne_base.source.tolist()+ligne_base.target.tolist()).items() if v==1]
    df_noeud_unique=pd.DataFrame(noeud_uniq, columns=['noeud_uniq'])
    lgn_tch_noeud_uniq=df_lignes.loc[((df_lignes['source'].isin(noeud_uniq)) | (df_lignes['target'].isin(noeud_uniq))) & (~df_lignes.index.isin(ligne_base.index.tolist()))].copy()
    # trouver les correspondance avec doublons de noeud possible et une ligne par corresp
    coresp_ligne_noeud=pd.concat(
        [lgn_tch_noeud_uniq.reset_index().merge(df_noeud_unique, left_on='source', right_on='noeud_uniq')[['id_ign','noeud_uniq']],
         lgn_tch_noeud_uniq.reset_index().merge(df_noeud_unique, left_on='target', right_on='noeud_uniq')[['id_ign','noeud_uniq']]],axis=0)
    #rapatriementdes ifos de longueur du tronc_elem
    coresp_te_noeud=coresp_ligne_noeud.merge(df_affectation, left_on='id_ign', right_on='id')[['id_ign','noeud_uniq','idtronc']].merge(tronc_elem_synth[
        ['long']],left_on='idtronc', right_index=True).rename(columns={'long' : 'long_te'})
    noeuds_fin=coresp_te_noeud.sort_values('noeud_uniq').groupby('noeud_uniq').agg({'idtronc':lambda x : tuple(set(x))})
    noeuds_fin=noeuds_fin.drop_duplicates('idtronc')

    # pour info si plus de 2 boeuds uniques alors il faut filtrer les non représentatif
    if len(noeuds_fin)>2 :
        coresp_te_noeud_filtre_lg=coresp_te_noeud.loc[coresp_te_noeud['long_te']>50] #on ne garde pas les tout petit tronc
        coresp_te_noeud_grp=coresp_te_noeud_filtre_lg.sort_values('noeud_uniq').groupby('noeud_uniq').agg(
            {'idtronc':lambda x : tuple(set(x)),'long_te':lambda x : tuple(set(x))})
        noeuds_fin=coresp_te_noeud_grp.loc[~coresp_te_noeud_grp.apply(lambda x : len(x['idtronc'])==1 ,axis=1)].copy()
        return noeuds_fin,coresp_te_noeud_grp, noeud_uniq # on pourrait creer une erreur pour ca
    return noeuds_fin,coresp_te_noeud, noeud_uniq

In [12]:
def tronc_elem_rdpt(num_rdpt, df_affectation,df_lignes):
    """
    Trouver les tronc_elem qui se rejoigne à un rd point, identifier par un noeud
    en entree : 
       num_rdpt : integer,  id_rdpt de carac_rdpt
       df_affectation : df de correspondance id_ign idtronc_elem, issu de regrouper_troncon
       df_lignes df des lignes bdtopo avec rd point et index=id_ign
    en sortie : 
        df_rdpt_tronc_elem : df avec en index le noeud caracteristique du rd et en data un tuple des tronc_elem arrivant
    """
    tronc_elem_entrant=df_affectation.loc[(df_affectation.id.isin(df_lignes.loc[list(carac_rd_pt.loc[num_rdpt].id_ign_entrant)].index.tolist())) 
                                              & (df_affectation['idtronc']!=num_tronc)].copy()#trouver les tronc_elem qui touchent
    #affectation arbitraire du rd poit à un noeud le composant 
    noeud_uniq_rdpt=df_lignes.loc[carac_rd_pt.loc[num_rdpt]['id_ign_rdpt'][0]]['source']
    tronc_elem_entrant['noeud_uniq']=noeud_uniq_rdpt
    #creer une df de mm structure qu noeuds_fin
    df_rdpt_tronc_elem=tronc_elem_entrant.groupby('noeud_uniq').agg({'idtronc':lambda x : tuple(set(x))})
    return df_rdpt_tronc_elem

In [67]:
def tronc_elem_rd_pt_global (nb_rdpt, ligne_base,num_rdpt, df_affectation,df_lignes) : 
    """
    adaptation de la fonction tronc_elem_rdpt selon le nombre de rdpt du tronc_elem
    en entree : 
        nb_rdpt : integer : nombre de rdpt significatifs
        ligne_base : df des lignes constituant le tronc elem. issus de df_lignes
        num_rdpt : list d'integer,  id_rdpt de carac_rdpt
        df_affectation : df de correspondance id_ign idtronc_elem, issu de regrouper_troncon
        df_lignes df des lignes bdtopo avec rd point et index=id_ign
    en sortie : 
        
    """
    if nb_rdpt==1: # dans ce cas il y a un rdpt qui ferme le troncon d'un coté. On commence par le coté ss rdpt
        noeuds_fin=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )[0]
        #puis le cote avec rd_pt : on cherche les troncs elem qui arriven
        df_rdpt_tronc_elem=tronc_elem_rdpt(num_rdpt[0], df_affectation,df_lignes)
        #agreg
        noeuds_fin=pd.concat([noeuds_fin, df_rdpt_tronc_elem], axis=0, sort=False)
    elif nb_rdpt==2 : # adns ce cas il faut trouver les correspondances pour chaque rd pt
        corresp1=tronc_elem_rdpt(num_rdpt[0], df_affectation,df_lignes)
        corresp2=tronc_elem_rdpt(num_rdpt[1], df_affectation,df_lignes)
        noeuds_fin=pd.concat([corresp1, corresp2],axis=0)
    return noeuds_fin

In [95]:
# à partir de df_affectation, je trouve les noeud qui ne sont présent qu'une fois
#regroupement des lignes par idtronc et creation des futurs attributs de src et tgt
group_idtronc=df_affectation.groupby('idtronc').agg({'id':lambda x : tuple(x)})
group_idtronc['src_elem']=-1
group_idtronc['tgt_elem']=-1
#pour une ligne : retrouver les lignes de base

for i,num_tronc in enumerate(df_affectation.idtronc.unique()) :
    if i%1000==0 : 
        print(f'nb lignes traitees {i}, heure : {datetime.now}, tronc :{num_tronc}')
    #num_tronc=68 #396 #3402 #206
    ligne_base=df_lignes.loc[list(group_idtronc.loc[num_tronc].id)]

    # Le pb c'est de gérer les rdpt qui termine un tronc elem. 
    #dc 1. combien de rd point dans le tronc eleme
    nb_rdpt=ligne_base.id_rdpt.nunique()

    #2. Gérer les différents cas du nb de rdpt 
    #si 0 : on va chercher les noeud qui n'apparaissent qu'une fois : ce son des fin de troncon elem
    if nb_rdpt==0 :
        noeuds_fin,synthese_croisement, noeud_uniq=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )
    else :  #on attaque les cas en fonction du nombre de rdpt   
        list_rdpt=ligne_base.loc[ligne_base['id_rdpt'].notna()].id_rdpt.unique()
    #si 1 : ce rd point est-il en fin de trocnon ou non. C'est tarduit par le nb de route différentes qui intersectenet les rd point (attribut de carac_rd_pt)
        if nb_rdpt==1 : 
            nb_voie_entrante=carac_rd_pt.loc[num_rdpt].nb_rte_rdpt
            if nb_voie_entrante==1 : #ds ce cas c'est comme si pas de rdpt
                noeuds_fin=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )[0]
            else : 
                noeuds_fin=tronc_elem_rd_pt_global(nb_rdpt, ligne_base,list_rdpt, df_affectation,df_lignes)         
        else : #si plusieurs rd pt
            #il faut connaitre le nb de rd pt avec plusieurs voie entrante. sinon le max est 2 si plus que 2 alors erreur
            tt_rdpt=carac_rd_pt.loc[list_rdpt] # tous les rd pt du tronc elem
            rd_pt_multi_voie=tt_rdpt.loc[tt_rdpt['nb_rte_rdpt']>1].index.tolist() #que les multi voies
            if len(rd_pt_multi_voie)==0 : # c'est le cas ss rdpt
                noeuds_fin=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )[0] 
            elif len(rd_pt_multi_voie)==1 : # si 1 seul on revient sur le cas du dessus,
                noeuds_fin=tronc_elem_rd_pt_global(1, ligne_base,rd_pt_multi_voie, df_affectation,df_lignes)
            elif len(rd_pt_multi_voie)==2 : # adns ce cas il faut trouver les correspondances pour chaque rd pt
                noeuds_fin=tronc_elem_rd_pt_global(2, ligne_base,rd_pt_multi_voie, df_affectation,df_lignes)
            else : #alors c'est une erreur
                print('erreur nb de rd point significatif, revoir le sectionnement')

    #mettre à jour la df créée au départ : 
    dico_attribut={0:'src_elem', 1:'tgt_elem'}
    for i in range(len(noeuds_fin)) : 
        if i >= 2 : 
            print(f'erreur sur troncon {num_tronc}, df noeud :  {noeuds_fin}')
        group_idtronc.loc[num_tronc, dico_attribut[i]]=noeuds_fin.index.values[i] #mise à jour des valuers de pour le troncon de base
        group_idtronc.loc[num_tronc, dico_attribut[i]]=noeuds_fin.index.values[i]
    for num_tronc_sup in [x for y in noeuds_fin.idtronc.tolist() for x in y] : #mise à jour des valeusr pour les autres troncons
        for i in range(len(noeuds_fin)) :
            if group_idtronc.loc[num_tronc_sup][dico_attribut[0]]==-1 : 
                group_idtronc.loc[num_tronc_sup, dico_attribut[0]]=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
                break
            elif group_idtronc.loc[num_tronc_sup][dico_attribut[1]]==-1 :
                group_idtronc.loc[num_tronc_sup, dico_attribut[1]]=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
                break
            else : 
                print(f'{num_tronc_sup} deja renseigne' )

nb lignes traitees 0, heure : <built-in method now of type object at 0x00007FFEAC903D60>, tronc :0
5 deja renseigne
5 deja renseigne
8 deja renseigne
8 deja renseigne
erreur sur troncon 6337, df noeud :                   idtronc                                  long_te
noeud_uniq                                                       
66001       (11016, 500)  (353.78757309698466, 5467.885551698945)
72616       (10875, 500)   (5467.885551698945, 309.6104307447416)
73237       (1892, 8853)   (3097.05124851706, 423.14520467961364)
83740       (2354, 1892)    (3097.05124851706, 361.3033189049768)


KeyError: 2

In [82]:
noeuds_fin

,idtronc
noeud_uniq,
72297,"(4504, 3713)"
69869,"(2741, 3005)"


In [47]:
noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)

noeud_uniq
39594     True
52060    False
dtype: bool

In [20]:
synthese_croisement.loc[~synthese_croisement.apply(lambda x : len(x['idtronc'])==1 ,axis=1)].copy()

,idtronc,long_te
noeud_uniq,,
9466,"(938, 1563, 12422)","(445.15375219309715, 1054.161900553276, 431.0587952118286)"


In [328]:
#mettre à jour la df créée au départ : 
group_idtronc.loc[num_tronc, 'src_elem']=noeuds_fin.index.values[0] #mise à jour des valuers de pour le troncon de base
group_idtronc.loc[num_tronc, 'tgt_elem']=noeuds_fin.index.values[1]
for num_tronc_sup in [x for y in noeuds_fin.idtronc.tolist() for x in y] : #mise à jour des valeusr pour les autres troncons
    if group_idtronc.loc[num_tronc_sup].src_elem==-1 : 
        group_idtronc.loc[num_tronc_sup, 'src_elem']=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
    elif group_idtronc.loc[num_tronc_sup].tgt_elem==-1 :
        group_idtronc.loc[num_tronc_sup, 'tgt_elem']=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
    else : 
        print(f'{num_tronc_sup} deja renseigne' )

In [200]:
df_affectation.idtronc.unique()

3639